In [1]:
import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf

tf.keras.backend.set_floatx('float32')

2023-03-01 23:54:09.032167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 23:54:09.395446: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-01 23:54:10.706628: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/leonkl/anaconda3/envs/LAB/lib/python3.9/site-packages/cv2/../../lib64:
2023-03-01

In [2]:
train_index = ["A-1"]#, "B-1", "C-1", "D-1", "E-1"]
path_cores = "TMA_cores_M06_M07_panels/M06/Cores/"
path_mxIF = "Texts_small_coregistered/"

In [3]:
train_cores = [cv.imread(path_cores + index + ".png") for index in train_index]
train_mxIF = [pd.read_csv(path_mxIF + index + ".csv") for index in train_index]

In [28]:
BUFFER = 50
BATCH = 8
CELL_SIZE = (32, 32)
MXIF_FEATURES = ["Cell X Position", "Cell Y Position"]

In [51]:
def data_generator():
    for i in range(len(train_index)):
        X = list(train_mxIF[i].loc[:,'Cell X Position'])
        Y = list(train_mxIF[i].loc[:,'Cell Y Position'])

        for j,(x,y) in enumerate(zip(X,Y)):
            if np.isnan(x) or np.isnan(y):
                continue
            if round(x - CELL_SIZE[0]) < 0 or round(x + CELL_SIZE[0]) > train_cores[i].shape[1]:
                continue
            if round(y < CELL_SIZE[1]) < 0 or round(y + CELL_SIZE[1]) > train_cores[i].shape[0]:
                continue

            cell_image = train_cores[i][round(y-CELL_SIZE[1]):round(y+CELL_SIZE[1]),
                                        round(x-CELL_SIZE[0]):round(x+CELL_SIZE[0])] / 255
            
            cell_features = np.array(train_mxIF[i].loc[j, MXIF_FEATURES], dtype=np.float32)
            yield (cell_image, cell_features)


In [52]:
train_ds = tf.data.Dataset.from_generator(data_generator,
                    output_signature=(tf.TensorSpec(shape=(2*CELL_SIZE[1],2*CELL_SIZE[0],3), dtype=tf.float32),
                                      tf.TensorSpec(shape=(len(MXIF_FEATURES)), dtype=tf.float32)))
train_ds = train_ds.shuffle(BUFFER)
train_ds = train_ds.batch(BATCH)

In [57]:
class DoubleAutoencoder(tf.keras.models.Model):
    def __init__(self, latent_dim=(48, 16)):
        super(DoubleAutoencoder, self).__init__()
        self.latent_dim_conv = latent_dim[0]
        self.latent_dim_fnn = latent_dim[1]
        
        self.encoder_conv = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(2*CELL_SIZE[1], 2*CELL_SIZE[0], 3)),
            tf.keras.layers.Conv2D(filters=6, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(filters=12, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(filters=18, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(filters=24, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(filters=36, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(filters=48, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(self.latent_dim_conv)])

        self.decoder_conv = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(self.latent_dim_conv)),
            tf.keras.layers.Dense(units=48, activation='relu'),
            tf.keras.layers.Reshape(target_shape=(1, 1, 48)),
            tf.keras.layers.Conv2DTranspose(filters=36, kernel_size=3, 
                                            strides=(2, 2), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2DTranspose(filters=24, kernel_size=3, 
                                            strides=(2, 2), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2DTranspose(filters=18, kernel_size=3, 
                                            strides=(2, 2), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2DTranspose(filters=12, kernel_size=3, 
                                            strides=(2, 2), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2DTranspose(filters=6, kernel_size=3, 
                                            strides=(2, 2), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, 
                                            strides=(2, 2), padding='same')])
        
        self.encoder_fnn = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(len(MXIF_FEATURES))),
            tf.keras.layers.Dense(units=64, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(units=32, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(units=self.latent_dim_fnn, activation='relu')
        ])

        self.decoder_fnn = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(self.latent_dim_fnn)),
            tf.keras.layers.Dense(units=32, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(units=64, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(units=len(MXIF_FEATURES), activation='relu')
        ])

    def call(self, inputs):
        he_latent = self.encoder_conv(inputs[0])
        mxIF_latent = self.encoder_fnn(inputs[1])
        he = self.decoder_conv(he_latent)
        mxIF = self.decoder_fnn(mxIF_latent)
        return [he, mxIF]


In [58]:
autoencoder = DoubleAutoencoder()
for elem in train_ds.take(1):
    elem = (elem[0][:,:,:], elem[1][:,:])
    autoencoder(elem)
autoencoder.summary()

Model: "double_autoencoder_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_68 (Sequential)  (None, 48)                32850     
                                                                 
 sequential_69 (Sequential)  (None, 64, 64, 3)         32805     
                                                                 
 sequential_70 (Sequential)  (None, 16)                3184      
                                                                 
 sequential_71 (Sequential)  (None, 2)                 3170      
                                                                 
Total params: 72,009
Trainable params: 71,241
Non-trainable params: 768
_________________________________________________________________
